In [3]:
import pandas as pd
import numpy as np

import matplotlib.pylab as plt
%matplotlib inline

from tqdm import tqdm

pd.set_option("display.max_rows", 10)
pd.set_option('display.max_columns', 1100)

## Предположения

- Наблюдения в логах получены независимо
- Хэдэры ботов получены перемешиванием лейблов существующих корректных юзер-агентов

### Посмотрим распределение юзер-агентов

на примере одного из лог файлов. Например 35-го по счету

In [4]:
from logParser import ParseLogsFromFolder

main_data, values_data, order_data = ParseLogsFromFolder('Logs/', 35, 36, only_order=False)

  0%|          | 0/127 [00:00<?, ?it/s]


In [5]:
df = pd.DataFrame(main_data)
# del(main_data)
print(df.shape)
distribution_frame = df.User_Agent.value_counts() / df.shape[0] 
distribution_frame.head()

(31300, 3)


Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)                                                0.119936
Mozilla/5.0 (compatible; MJ12bot/v1.4.7; http://mj12bot.com/)                                                   0.063898
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)                                         0.039617
Mozilla/5.0 (compatible; spbot/5.0.3; +http://OpenLinkProfiler.org/bot )                                        0.030863
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36    0.023514
Name: User_Agent, dtype: float64

### На основе новой выборки получим хэдэры с намешанными лэйблами

с полученным выше распределением юзер-агентов

In [6]:
from logParser import ParseLogsFromFolder

main_data, values_data, order_data = ParseLogsFromFolder('Logs/', 36, 37, only_order=False)

  0%|          | 0/127 [00:00<?, ?it/s]


In [7]:
main = pd.DataFrame(main_data)
print(df.shape)
main.User_Agent

(31300, 3)


0                              Opera/8.01 (Windows NT 5.1)
1                              Opera/8.01 (Windows NT 5.1)
2                              Opera/8.01 (Windows NT 5.1)
3        Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
4        Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/2...
                               ...                        
13514    Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:5...
13515    Mozilla/5.0 (compatible; bingbot/2.0; +http://...
13516    Mozilla/5.0 (compatible; YandexMetrika/2.0; +h...
13517    Mozilla/5.0 (compatible; Googlebot/2.1; +http:...
13518    Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
Name: User_Agent, Length: 13519, dtype: object

In [8]:
np.cumsum(distribution_frame)

Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)                                                                                 0.119936
Mozilla/5.0 (compatible; MJ12bot/v1.4.7; http://mj12bot.com/)                                                                                    0.183834
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)                                                                          0.223450
Mozilla/5.0 (compatible; spbot/5.0.3; +http://OpenLinkProfiler.org/bot )                                                                         0.254313
Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36                                     0.277827
                                                                                                                                                   ...   
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.4 (KHTML, like Gecko) Chrome/22

In [9]:
main.shape

(13519, 3)

In [10]:
x = np.random.rand()
print(x)
print((np.cumsum(distribution_frame) > x).tail())
distribution_frame[np.cumsum(distribution_frame) > x].index[0]

0.83382209303744
Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.4 (KHTML, like Gecko) Chrome/22.0.1104.222 YaBrowser/1.5.1104.222 Safari/537.4                      True
Mozilla/5.0 (Linux; Android 4.4.4; Ontario_2 Build/I6) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/33.0.0.0 Safari/537.36          True
Mozilla/5.0 (Linux; Android 5.0.1; K019 Build/LRX22C) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.91 Safari/537.36                   True
Mozilla/5.0 (Linux; Android 4.1.2; LG-P765 Build/JZO54K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Mobile Safari/537.36         True
Mozilla/5.0 (Linux; Android 5.1; ZTE Blade A476 Build/LMY47D) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.91 Mobile Safari/537.36    True
Name: User_Agent, dtype: bool


'Mozilla/5.0 (Linux; Android 5.0.2; Lenovo S90-A Build/LRX22G) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Mobile Safari/537.36'

In [11]:
distribution_frame.sum()

0.9948881789137841

In [12]:
sample_user_agents = []
norm_koef = distribution_frame.sum()
for sample_index in range(main.shape[0]):
    uniform_value = np.random.rand() * norm_koef 
#     print (uniform_value)
    sample_user_agents.append(
        distribution_frame[np.cumsum(distribution_frame) > uniform_value].index[0])
sample_user_agents = pd.Series(sample_user_agents)
sample_user_agents.value_counts() / main.shape[0]

Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)                                                                                             0.122642
Mozilla/5.0 (compatible; MJ12bot/v1.4.7; http://mj12bot.com/)                                                                                                0.060507
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)                                                                                      0.037207
Mozilla/5.0 (compatible; spbot/5.0.3; +http://OpenLinkProfiler.org/bot )                                                                                     0.032843
Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)                                                                                     0.023301
                                                                                                                                                               ...   
Mozi

In [13]:
main.User_Agent = sample_user_agents
main.head()

,User_Agent,id,ip
0,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,1489183203,37.115.191.193
1,Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKi...,1489183204,37.115.191.193
2,Mozilla/4.0 (compatible; MSIE 6.0; America Onl...,1489183205,37.115.191.193
3,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,1489183220,93.158.152.7
4,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...,1489183229,45.62.243.135


Как видим в целом распределение юзер-агентов как мы и хотели

Обобщим в функцию и добавим в logParser

In [14]:
def GetBotSample(distr_log_index_begin, distr_log_index_end, 
                 base_log_index_begin, base_log_index_end):
    main_data, _, _ = ParseLogsFromFolder(
        'Logs/', distr_log_index_begin, distr_log_index_end, only_order=False)
    df = pd.DataFrame(main_data)
    del(main_data)
    distribution_frame = df.User_Agent.value_counts() / df.shape[0] 
    cumulative_frame = np.cumsum(distribution_frame)
    
    main_data, values_data, order_data = ParseLogsFromFolder(
        'Logs/', base_log_index_begin, base_log_index_end, only_order=False)
    main_df = pd.DataFrame(main_data)
    
    sample_user_agents = []
    norm_koef = distribution_frame.sum()
    for sample_index in range(main_df.shape[0]):
        uniform_value = np.random.rand() * norm_koef 
        sample_user_agents.append(
            distribution_frame[cumulative_frame > uniform_value].index[0])
    sample_user_agents = pd.Series(sample_user_agents)
    
    main_df.User_Agent = sample_user_agents
    
    return main_df, values_data, order_data

In [15]:
main_bot_data, values_bot_data, order_bot_data = GetBotSample(36, 37, 37, 38)
main_bot_data

  0%|          | 0/127 [00:00<?, ?it/s]


,User_Agent,id,ip
0,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,1489269605,42.236.99.58
1,Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/2...,1489269610,2.94.54.222
2,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,1489269637,95.56.103.103
3,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,1489269668,165.231.0.242
4,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...,1489269713,66.102.9.142
...,...,...,...
16770,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1489362932,93.158.152.7
16771,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,1489362985,46.229.168.71
16772,Mozilla/5.0 (compatible; bingbot/2.0; +http://...,1489363048,93.158.152.7
16773,Mozilla/5.0 (compatible; AhrefsBot/5.2; +http:...,1489363153,46.229.168.74
